In [ ]:
import sys
sys.path.append("../src/initialization/")
import setup_db

setup_db.setup_db_titanic()

In [ ]:
# Connect to Postgres
import os
import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine(
            "postgresql+psycopg2://"
            + os.getenv("SQL_USER")
            + ":"
            + os.getenv("SQL_PASSWORD")
            + "@"
            + os.getenv("SQL_HOST")
            + ":"
            + os.getenv("SQL_PORT")
            + "/"
            + os.getenv("SQL_DB")
        )
titanic_data = pd.read_sql_table("titanic", engine)
titanic_data

In [ ]:
# Connect to PySpark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("TitanicExample") \
    .config("spark.jars", "/srv/KaggleChallenge/plugins/postgresql-42.5.1.jar") \
    .getOrCreate()

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://" + os.getenv("SQL_HOST") +":" + os.getenv("SQL_PORT") + "/" + os.getenv("SQL_DB")) \
    .option("dbtable", "titanic") \
    .option("user", os.getenv("SQL_USER")) \
    .option("password", os.getenv("SQL_PASSWORD")) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

In [ ]:
# Connect to mlflow
import mlflow

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("my-experiment")


In [ ]:
# Connect to Kafka
import kafka

In [ ]:
# Connect to MinIO
import minio
from minio import Minio

# Create a client with the MinIO server playground, its access key
# and secret key.
client = Minio(
    endpoint="minio:9000",
    access_key="GArH6uh55RJ44MBc",
    secret_key="wZki2TznKq0w39wprAqm7KvKCBPeH5nm",
    secure=False
)

found = client.bucket_exists("titanic")
if not found:
    client.make_bucket("titanic")
else:
    print("Bucket 'titanic' already exists")

client.fput_object(
    "titanic", "00_Intro.ipynb", "/srv/KaggleChallenge/extra_data/00_Intro.ipynb",
)
print(
    "'00_intro.ipynb' is successfully uploaded as "
    "object 00_intro.ipynb' to bucket '00_intro.ipynb'."
)

In [ ]:
#import logging
#logging.basicConfig(level=logging.DEBUG)

In [ ]:

from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='kafka:9092')
for _ in range(10):
    producer.send('foobar', b'some_message_bytes')

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer('my_favorite_topic', bootstrap_servers='kafka:9092')
for msg in consumer:
    print (msg)